# Data Reading

In [ ]:
import pandas as pd


df = pd.read_csv("data/funda-rotterdam-11-18-2024.csv")

In [ ]:
df.head()

# Data Preparation

Let's first check the shape of the DataFrame:

In [ ]:
df.shape

There are 4650 samples and 33 columns

## Standardize Column Names

In [ ]:
df.columns

Two changes are made. The period `.` is replaced by `_` such that we can make use of attribute-style access to column names, and the term `blikvanger` is replaced by its English translation, which is `eyecatcher`:

In [ ]:
df.columns = (
    df.columns
    .str.replace(".", "_")
    .str.replace("blikvanger", "eyecatcher")
)

## Filtering Data

Not all the data we have is of interest. There is one column `object_type`, which contains information on what kind of property is being sold:

In [ ]:
df.object_type.unique()

For this project we are only interested in apartments and houses, so we will filter out the rest:

In [ ]:
df = df[df.object_type.isin(["apartment", "house"])]

In [ ]:
df.shape

This filtered out 88 samples.

There is another column `object_type_specifications_house_type` which specifies what type of house it is. It has the following values:

We are not interested in boats:

In [ ]:
df = df[df.object_type_specifications_house_type!="house_boat"]

## Check for missing values

Let's first check the percentage of missing values per column:

In [ ]:
df.isna().mean().sort_values(ascending=False)

In [ ]:
import numpy as np
import ast

def list_to_ohe(df, column):
    try:
        return (
            df
            .assign(**{f"{column}_{v}": df[column].apply(ast.literal_eval).apply(lambda x: v in x) for v in set(df[column].apply(ast.literal_eval).sum())})
            .drop(columns=column)
        )
    except ValueError:
        return (
            df
            .assign(**{f"{column}_{v}": df[column].apply(lambda x: v in x)  for v in set(df[column].sum())})
            .drop(columns=column)
        )

df = (
    df
    .assign(
        object_type_specifications_appartment_type=df.object_type_specifications_appartment_type.fillna("not_applicable").astype("string"),
        object_type_specifications_house_type=df.object_type_specifications_house_type.fillna("not_available").astype("string"),
        object_type_specifications_house_orientation=df.object_type_specifications_house_type.fillna("not_available").astype("string"),
        exterior_space_garden_size=pd.to_numeric(df.exterior_space_garden_size.fillna(0), downcast="integer"),
        publish_date=pd.to_datetime(df.publish_date, format="mixed"),
        floor_area=pd.to_numeric(df.floor_area.str.extract(r"\[(\d+)\]")[0], downcast="integer"),
        plot_area=pd.to_numeric(df.plot_area.str.extract(r"\[(\d+)\]")[0], downcast="integer"),
        number_of_rooms=pd.to_numeric(df.number_of_rooms, downcast="integer"),
        number_of_bedrooms=pd.to_numeric(df.number_of_bedrooms, downcast="integer"),
        offering_type=df.offering_type.str.extract(r"\['([a-z]+)'\]")[0].astype("string"),
        garage_total_capacity=df.garage_total_capacity.fillna(0),
        price_selling_price=pd.to_numeric(df.price_selling_price.str.extract(r"\[(\d+)\]")[0], downcast="integer"),
    )
    .query('''price_selling_price > 1e5''')
    # .assign(price_selling_price=lambda df_: df_.price_selling_price.apply(np.log1p))
    .pipe(list_to_ohe, column="exterior_space_type")
    .pipe(list_to_ohe, column="exterior_space_garden_orientation")
    .pipe(list_to_ohe, column="surrounding")
    .pipe(list_to_ohe, column="garage_type")
    .pipe(list_to_ohe, column="amenities")
    .pipe(list_to_ohe, column="accessibility")
    .astype({
        "address_municipality": "string",
        "price_selling_price_condition": "string",
        "construction_type": "string",
        "construction_period": "string",
        "object_type": "string",
        "energy_label": "string",
    })
    .query('''offering_type=="buy"''')
    .drop(
        columns=[
            "publish_date",
            "address_neighbourhood", 
            "address_wijk", 
            "address_province", 
            "address_street_name",
            "eyecatcher_text",
            "project_url",
            "offering_type",
        ]
    )
)

In [ ]:
from sklearn.model_selection import train_test_split

df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

In [ ]:
df_full_train = df_full_train.reset_index(drop=True)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [ ]:
y_train = df_train.price_selling_price.values
y_val = df_val.price_selling_price.values
y_test = df_test.price_selling_price.values

In [ ]:
from sklearn.feature_extraction import DictVectorizer

dicts_train = df_train.drop(columns="price_selling_price").to_dict(orient="records")
dicts_val = df_val.drop(columns="price_selling_price").to_dict(orient="records")
dv = DictVectorizer(sparse=False)
dv.fit(dicts_train)

X_train = dv.transform(dicts_train)
X_val = dv.transform(dicts_val)

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)
y_pred_train = model.predict(X_train)
y_pred_val = model.predict(X_val)

In [ ]:
from sklearn.metrics import root_mean_squared_error

rmse_train = root_mean_squared_error(y_train, y_pred_train)
rmse_val = root_mean_squared_error(y_val, y_pred_val)

print(rmse_train)
print(rmse_val)